In [17]:
import tensorflow as tf
import numpy as np
import pytest

from helmo.util.parallel import in_separate_process
import helmo.util.tensor_ops as tensor_ops

@in_separate_process
def test_raises_if_numdims_small_tensor():
    with tf.Session() as sess:
        numdims = 1
        tens = [[2]]
        expanded = tensor_ops.expand_multiple_dims(tens, numdims, [1, 2])
        with pytest.raises(tf.errors.InvalidArgumentError):
            r = sess.run(expanded)


test_raises_if_numdims_small_tensor()



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Process Process-2:
Traceback (most recent call last):
  File "/home/anton/.local/lib/python3.5/site-packages/_pytest/python_api.py", line 729, in __exit__
    fail(self.message)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/anton/h-elmo/helmo/util/parallel.py", line 14, in target_function
    res = func(*args, **kwargs)
  File "<ipython-input-17-e8bf7dd0e7f6>", line 15, in test_raises_if_numdims_small_tensor
    r = sess.run(expanded)
  File "/home/anton/.local/lib/python3.5/site-packages/_pytest/outcomes.py", line 117, in fail
    raise Failed(msg=msg, pytrace=pytrace)
Failed: DID NOT RAISE <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>


KeyboardInterrupt: 

In [1]:
import tensorflow as tf


def get_all_values_except_specified(tensor, excluded):
    with tf.name_scope('get_all_values_except_specified'):
        tensor = tf.reshape(tensor, [-1])
        excluded = tf.reshape(excluded, [-1])
        excluded_shape = tf.shape(excluded)
        tensor_expanded = tf.reshape(tensor, [-1, 1])
        multiples = tf.concat([[1], excluded_shape], 0)
        tensor_expanded = tf.tile(tensor_expanded, multiples)
        masks = tf.cast(tf.equal(tf.cast(tensor_expanded, tf.int32), tf.cast(excluded, tf.int32)), tf.int32)
        mask = tf.reduce_sum(masks, [1])
        mask = tf.cast(tf.cast(mask, dtype=tf.bool), dtype=tf.int32) - 1
        return tf.boolean_mask(tensor, mask)
    

tensor, num_dims, axes, output = (
                    [[[1, 2], [3, 4]], [[5, 6], [7, 8]]],

                    5,

                    [0, 2, 4],

                    [[[[[1, 2]], [[3, 4]]], [[[5, 6]], [[7, 8]]]]],
                )

if not tf.contrib.framework.is_tensor(tensor):
    tensor = tf.constant(tensor)
if not tf.contrib.framework.is_tensor(axes):
    axes = tf.constant(axes, dtype=tf.int32)
sh = tf.shape(tensor, out_type=tf.int32)
nd = tf.shape(sh, out_type=tf.int32)[0]
assert_axes_smaller_than_num_dims = tf.assert_less(
    axes, num_dims, message='`axes` has to be less than `num_dims`')
check_num_dims = tf.assert_greater_equal(
    num_dims, nd,
    message='`num_dims` has to be greater or equal to number of dimensions in `tensor`'
)
ass_axes_bigger_or_equal_than_num_dims = tf.assert_greater_equal(axes, -num_dims)

negative_axes_mask = tf.cast(axes < 0, tf.int32)
axes += negative_axes_mask * num_dims

ones_for_expansion = tf.ones(tf.reshape(num_dims - nd, [1]), dtype=tf.int32)
shape_for_expansion = tf.concat([sh, ones_for_expansion], 0)

tensor = tf.reshape(tensor, shape_for_expansion)

# remained_axes = get_all_values_except_specified(tf.range(num_dims, dtype=tf.int32), axes)
# perm = tf.concat([axes, remained_axes], 0)
updates = tf.range(0, num_dims, 1, dtype=tf.int32)
remained_positions = get_all_values_except_specified(tf.range(num_dims, dtype=tf.int32), axes)
indices = tf.concat([axes, remained_positions], 0)
indices = tf.reshape(indices, [-1, 1])
perm_shape = tf.reshape(num_dims, [1])
perm = tf.scatter_nd(indices, updates, perm_shape)

with tf.control_dependencies([check_num_dims, assert_axes_smaller_than_num_dims, ass_axes_bigger_or_equal_than_num_dims]):
    tensor = tf.transpose(tensor, perm=perm)
    

with tf.Session() as sess:
    print(sess.run(tensor))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

[[[[[1 2]]

   [[3 4]]]]



 [[[[5 6]]

   [[7 8]]]]]


In [2]:
import tensorflow as tf

import helmo.util.tensor_ops as ops

tensor = tf.constant([[[1, -1], [-1, 1]], [[1, 1], [-1, -1]]])
cor_axis = -1
reduced_axes = [1]

corrv = ops.get_correlation_values(tensor, reduced_axes, cor_axis)
corr = ops.correlation(tensor, reduced_axes, cor_axis)

with tf.Session() as sess:
    v, c = sess.run([corrv, corr])
    print(v)
    print(c)

[[-1.00000012  1.00000012]]
[[[ 1.00000012 -1.00000012]
  [-1.00000012  1.00000012]]

 [[ 1.00000012  1.00000012]
  [ 1.00000012  1.00000012]]]


In [3]:
import queue
import Multiprocessing

def target_function(q, *args, **kwargs):
    print("in target function")
    res = func(*args, **kwargs)
    q.put(res)
    print("res is put in queue")

def wrapper(*args, **kwargs):
    print("in wrapper")
    q = queue.Queue()
    p = multiprocessing.Process(target=target_function, args=tuple([q]+list(args)), kwargs=kwargs)
    p.start()
    res = q.get()
    print("got result")
    p.join()
    print("joined")
    return res

def func():
    return 1

wrapper()

in wrapper


NameError: name 'queue' is not defined

In [13]:
import tensorflow as tf

print(tf.placeholder(tf.float32).__class__.__bases__)
print(tf.zeros([1]).__class__.__bases__)
print(tf.Variable(0).__class__.__bases__[0].__bases__[0].__bases__)

(<class 'tensorflow.python.framework.ops._TensorLike'>,)
(<class 'tensorflow.python.framework.ops._TensorLike'>,)
(<class 'tensorflow.python.training.checkpointable.base.CheckpointableBase'>,)
